In [1]:
import os
import sqlite3
import pandas as pd

In [2]:
## Reading data, creating a sqlite db and creating initial dataframe with all games

# Set folder to use
folder_path = '../2023_CFB_Data_Play/' 

In [3]:
# file count
file_count = 0

# Dictionary for dataframes from csv files
dataframe_dictionary = {}

# Loop through folder of files, if csv
for file_name in os.listdir(folder_path):
    # print(file_name)
    if file_name.endswith('.csv'):
        file_count = file_count + 1
        file_path = os.path.join(folder_path, file_name)
        dataframe_dictionary[file_count] = file_path
    else:
        print(f"Not a csv: {file_path}")

# print(dataframe_dictionary)

In [4]:
print(dataframe_dictionary)

{1: '../2023_CFB_Data_Play/MichState_2023.csv', 2: '../2023_CFB_Data_Play/Illinois_2023.csv', 3: '../2023_CFB_Data_Play/Maryland_2023.csv', 4: '../2023_CFB_Data_Play/Northwestern_2023.csv', 5: '../2023_CFB_Data_Play/Purdue_2023.csv', 6: '../2023_CFB_Data_Play/LaTech_2023.csv', 7: '../2023_CFB_Data_Play/Colorado_2023.csv', 8: '../2023_CFB_Data_Play/Northern_Illinois_2023.csv', 9: '../2023_CFB_Data_Play/Minnesota_2023.csv', 10: '../2023_CFB_Data_Play/Wisconsin_2023.csv', 11: '../2023_CFB_Data_Play/Iowa_2023.csv', 12: '../2023_CFB_Data_Play/Michigan_2023.csv'}


In [5]:
# create list to hold dataframes as read in via csv parser
dataframes = []

# loop through csv files, read and add to list (check for date format)
for count, file_path in dataframe_dictionary.items():
    try:
        df = pd.read_csv(file_path)
        dataframes.append(df)
    except Exception as e: 
        print(f'Can note read: {file_path}')

In [6]:
# Length should be the same and number of games / files
print(len(dataframes))

12


In [7]:
# combine all the dataframes from list into one df
combined_df = pd.concat(dataframes, ignore_index=True)

In [8]:
# The total number of plays for season and season dataframe
print(len(combined_df))

1062


In [9]:
## Use the dataframe and create a sqlite db

connection = sqlite3.connect('../2023_SqliteDV.db')

In [10]:
combined_df.to_sql('plays', connection, if_exists='replace', index=False)

1062

In [11]:
connection.close()

In [76]:
## Start some analysis

# Question 1: What were the distribution of yards by rushing attempt for Emmett Johnson

# List of play types to include in rushing dataframe
rushing_types = ['Rush', 'Rushing Touchdown', 'Sack', 'Fumble Recovery (Opponent)']

# Filter the DataFrame to include only Rush play types
rush_df = combined_df[combined_df['Play Type'].isin(rushing_types)]

In [77]:
print(len(rush_df))

482


In [78]:
# Filter the DataFrame to include only rows with 'Emmett Johnson' in the play text column
johnson_df = rush_df[rush_df['Play Text'].str.contains('Emmett Johnson')]

# Adding 'Anthony Grant' as a check
grant_df = rush_df[rush_df['Play Text'].str.contains('Anthony Grant')]

In [79]:
johnson_total_yards = johnson_df['Yards Gained'].sum()
grant_total_yards = grant_df['Yards Gained'].sum()

In [80]:
print(johnson_total_yards)
print(grant_total_yards)

421
428


In [81]:
print(len(johnson_df))
print(len(grant_df))

88
100


In [18]:
# Emmett Johnson had 90 attempts and 411 rushing yards in 2023 according to espn
# Anthony Grant had 104 attempts and 413 rushing yards in 2023 according to espn
# https://www.espn.com/college-football/stats/player/_/view/offense/stat/rushing/group/5/table/rushing/sort/rushingYards/dir/desc

In [84]:
johnson_by_game = johnson_df.groupby('Defense').size()

In [85]:
print(johnson_by_game)

Defense
Illinois           7
Iowa              11
Louisiana Tech     2
Maryland          16
Michigan           2
Michigan State    13
Northwestern      12
Purdue            13
Wisconsin         12
dtype: int64


In [86]:
# Wisconsin and Maryland games are all short by 1 carry/play
# Unclear as going through the play by play with Wisconsin on ESPN, it only has 12 plays but stats show 13 carries

In [87]:
emmett_play = combined_df[combined_df['Play Text'].str.contains('Emmett Johnson')]

In [88]:
wisconsin_emmett = emmett_play[emmett_play['Defense'] == 'Wisconsin']

In [89]:
print(len(wisconsin_emmett))

12


In [90]:
wisconsin_emmett_rush = wisconsin_emmett[wisconsin_emmett['Play Type'] == 'Rush']

In [91]:
print(len(wisconsin_emmett_rush))

12


In [92]:
emmett_wis_total = wisconsin_emmett_rush['Yards Gained'].sum()
print(emmett_wis_total)

53


In [93]:
# The stats for Emmett according to ESPN are 13 carries and 50 yards
# So looking for a -3 yards play 

In [94]:
negative_three = combined_df[combined_df['Yards Gained'] == -3]

In [95]:
# print(negative_three)
# No plays for -3 yards by Emmett in the Wisconsin game?

In [96]:
maryland_emmett = emmett_play[emmett_play['Defense'] == 'Maryland']

In [97]:
print(len(maryland_emmett))

20


In [98]:
maryland_emmett_rush = maryland_emmett[maryland_emmett['Play Type'].isin(rushing_types)]

In [99]:
chosen_columns = ['Play Number', 'Defense', 'Yards Gained', 'Play Type', 'Play Text', 'Wallclock']

In [100]:
print(len(maryland_emmett_rush))

16


In [101]:
emmett_maryland_total = maryland_emmett_rush['Yards Gained'].sum()
print(emmett_maryland_total)

89


In [102]:
# ESPN shows 17 rushes for 84 yards
# 1 carry less, 5 yards more?

In [103]:
print(maryland_emmett_rush[chosen_columns].sort_values(by='Wallclock'))

     Play Number   Defense  Yards Gained Play Type  \
197            2  Maryland            10      Rush   
201            6  Maryland             4      Rush   
202            7  Maryland             1      Rush   
208            2  Maryland             3      Rush   
209            3  Maryland            10      Rush   
215            2  Maryland             5      Rush   
216            3  Maryland             6      Rush   
222            9  Maryland             7      Rush   
224           11  Maryland             1      Rush   
241            3  Maryland             3      Rush   
242            4  Maryland             2      Rush   
258            1  Maryland             2      Rush   
261            4  Maryland             2      Rush   
262            5  Maryland             2      Rush   
265            8  Maryland            29      Rush   
268           11  Maryland             2      Rush   

                                             Play Text  \
197  Emmett Johnson run

In [104]:
# Unclear as going through the play by play with Maryland on ESPN, it only has 16 plays but stats show 17 carries
# Same issue as noticed in Wisconsn game. Some play type is counting as a rush that is resulting in negative yards
# Not finding those plays yet.

In [105]:
# Get all rushing plays by Nebraska in Purdue game

purdue_rushing = rush_df[rush_df['Defense'] == 'Purdue']
print(len(purdue_rushing))

46


In [106]:
# Group all plays in Purdue game grouped by play type
# game id = 401520355

In [107]:
purdue_game_plays = combined_df[combined_df['Game Id'] == 401520355]

In [108]:
grouped_plays_purdue = purdue_game_plays.groupby('Play Type').size()

In [109]:
print(grouped_plays_purdue)

Play Type
End Period                     1
End of Game                    1
End of Half                    1
Field Goal Good                1
Fumble Recovery (Opponent)     2
Fumble Return Touchdown        1
Kickoff                        4
Kickoff Return (Offense)       2
Pass Incompletion              5
Pass Reception                 3
Passing Touchdown              2
Penalty                        7
Punt                           5
Rush                          40
Rushing Touchdown              1
Sack                           3
Timeout                        7
dtype: int64


In [110]:
neb_off_purdue = purdue_game_plays[purdue_game_plays['Defense'] == 'Purdue']

In [111]:
print(len(neb_off_purdue))

86


In [112]:
# Checking all rushing plays for Purdue game
neb_off_purdue_rush = neb_off_purdue[neb_off_purdue['Play Type'].isin(rushing_types)]

In [113]:
print(len(neb_off_purdue_rush))

46


In [ ]:
# ESPN shows 48 rushes, so 2 plays off yet for the team

In [115]:
# Get total yards for rushing at Purdue
neb_purdue_rush_yards = neb_off_purdue_rush['Yards Gained'].sum()

In [116]:
print(neb_purdue_rush_yards)

182


In [ ]:
# Rush yards are 27 yards high as ESPN shows 155 total

In [118]:
# Put yards gained into a list
yards_purdue_list = list(neb_off_purdue_rush['Yards Gained'])

In [119]:
print(len(yards_purdue_list))

46


In [120]:
print(yards_purdue_list)

[8, -11, 5, 3, 3, 5, 2, 2, 3, 16, 6, 1, 4, 6, 0, 2, 15, 1, 6, 1, 1, 1, 8, 5, 6, 2, 0, -6, 8, 4, 2, -7, 4, 4, 2, 3, 4, 4, 8, 2, 2, 28, 2, 3, 8, 6]


In [121]:
print(sum(yards_purdue_list))

182


In [114]:
print(neb_off_purdue_rush[chosen_columns].sort_values(by='Wallclock'))

     Play Number Defense  Yards Gained                   Play Type  \
345            2  Purdue             8                        Rush   
347            4  Purdue           -11                        Sack   
349            1  Purdue             5                        Rush   
350            2  Purdue             3                        Rush   
351            3  Purdue             3                        Rush   
353            1  Purdue             5                        Rush   
354            2  Purdue             2                        Rush   
356            4  Purdue             2                        Rush   
357            5  Purdue             3                        Rush   
359            7  Purdue            16                        Rush   
362           10  Purdue             1                        Rush   
361            9  Purdue             6                        Rush   
366           14  Purdue             4                        Rush   
367           15  Pu